# MLQA Extract `Chinese` Keywords (Query/Context)

In [1]:
import re
import os, sys
import json
import numpy as np
from tqdm.auto import tqdm
from IPython.display import clear_output
from contextlib import contextmanager

# https://pypi.org/project/chinese-keybert/
from chinese_keybert import Chinese_Extractor

import jieba
import jieba.analyse

## Keywords for `Context`

In [2]:
file_path = './data/Context_ZH.json'

with open(file_path, 'r') as f:
    data_zh = json.load(f)

print(f"Total num: {len(data_zh)}")
data_zh[:5]

Total num: 4538


[{'id': 0,
  'title': '電動勢',
  'context': '在电路学里，电动势（英语：electromotive force，缩写为emf）表征一些电路元件供应电能的特性。这些电路元件称为「电动势源」。电化电池、太阳能电池、燃料电池、热电装置、发电机等等，都是电动势源。电动势源所供应的能量每单位电荷是其电动势。假设，电荷'},
 {'id': 1,
  'title': '楚河州',
  'context': '楚河州包括有整个楚河河谷及邻近的山脉与峡谷。河谷的黑土非常肥沃，而且被从楚河引来的河水灌溉着。当地的农业生产计有：小麦、玉蜀黍、甜菜、马铃薯、紫花苜蓿及各种不同品种的蔬菜及水果。在苏联统治期间，省内有不少农产品加工及其他工业，使省内涌现多个新市镇，如：托克马克、坎特（Kant）及卡拉巴尔塔（Kara-Balta）等。相对于国内其他省份，本州的人口成份比较复杂，计有：俄罗斯人、乌克兰人、东干人（中国回民的后裔）、朝鲜人及德国人等。'},
 {'id': 2,
  'title': '凱提文',
  'context': '凯提文(Kaithi，कैथी)，也叫做Kayathi或Kayasthi，是历史上的一种文字，曾广泛用于北印度，主要是以前的西北行省和Oudh（今天的北方邦）和比哈尔。它曾用于书写法律、行政和私人记录。Unicode技术委员会已经接受了在Unicode标准中编码凯提文的提案，范围是U+11080-110CF。'},
 {'id': 3,
  'title': '凱提文',
  'context': '用凯提文记录的文档可追溯到至少16世纪。这种文字广泛用在莫卧儿帝国期间。在1880年代英属印度期间，这种文字被认可为比哈尔邦法庭上的官方文字。尽管一般而言凯提文曾在某些地区比城文更加广泛使用，它现在已经失去了竞争力。'},
 {'id': 4,
  'title': '爱丽丝梦游仙境',
  'context': '第五章：毛毛虫的建议（Advice from a Caterpillar）爱丽丝见到一棵蘑菇，上面坐著一条蓝色的毛虫。他抽著水烟，向爱丽丝探问起来。爱丽丝回应他，自己正在个性转变期之中，时常心绪不宁，她甚至连一首诗都记不起来。毛虫离开之前，告诉了她蘑菇的秘密：吃其中一半会使她变高，吃另一半会使她变矮。于是，她

In [3]:
kw_extractor = Chinese_Extractor()

/Users/zhizhang/anaconda3/envs/NLP/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
contexts = [context['context'] for context in data_zh]
contexts[:5]

['在电路学里，电动势（英语：electromotive force，缩写为emf）表征一些电路元件供应电能的特性。这些电路元件称为「电动势源」。电化电池、太阳能电池、燃料电池、热电装置、发电机等等，都是电动势源。电动势源所供应的能量每单位电荷是其电动势。假设，电荷',
 '楚河州包括有整个楚河河谷及邻近的山脉与峡谷。河谷的黑土非常肥沃，而且被从楚河引来的河水灌溉着。当地的农业生产计有：小麦、玉蜀黍、甜菜、马铃薯、紫花苜蓿及各种不同品种的蔬菜及水果。在苏联统治期间，省内有不少农产品加工及其他工业，使省内涌现多个新市镇，如：托克马克、坎特（Kant）及卡拉巴尔塔（Kara-Balta）等。相对于国内其他省份，本州的人口成份比较复杂，计有：俄罗斯人、乌克兰人、东干人（中国回民的后裔）、朝鲜人及德国人等。',
 '凯提文(Kaithi，कैथी)，也叫做Kayathi或Kayasthi，是历史上的一种文字，曾广泛用于北印度，主要是以前的西北行省和Oudh（今天的北方邦）和比哈尔。它曾用于书写法律、行政和私人记录。Unicode技术委员会已经接受了在Unicode标准中编码凯提文的提案，范围是U+11080-110CF。',
 '用凯提文记录的文档可追溯到至少16世纪。这种文字广泛用在莫卧儿帝国期间。在1880年代英属印度期间，这种文字被认可为比哈尔邦法庭上的官方文字。尽管一般而言凯提文曾在某些地区比城文更加广泛使用，它现在已经失去了竞争力。',
 '第五章：毛毛虫的建议（Advice from a Caterpillar）爱丽丝见到一棵蘑菇，上面坐著一条蓝色的毛虫。他抽著水烟，向爱丽丝探问起来。爱丽丝回应他，自己正在个性转变期之中，时常心绪不宁，她甚至连一首诗都记不起来。毛虫离开之前，告诉了她蘑菇的秘密：吃其中一半会使她变高，吃另一半会使她变矮。于是，她把蘑菇一分为二，果然，吃其中一半使她矮小无比，吃另一半则令她的脖子增长。她的脑袋直达树丛之中，树上的鸽子甚至误以为她长长的脖子是一条毒蛇。经过一番努力，爱丽丝终于恢复原来的身高。她蹒跚地走，偶然进入了一个小庄园。同时，她又利用蘑菇调校最适合的身高。']

In [5]:
kw_extractor.generate_keywords(contexts[990], top_k=5, rank_methods="mmr")[0]

Inference: 100%|██████████| 1/1 [00:00<00:00,  5.93it/s]


['花漾年华', '法国', '歌曲', '青少年', '拒收']

**Define a context manager for temporarily suppressing outputs**

In [6]:
data_zh[:5]

[{'id': 0,
  'title': '電動勢',
  'context': '在电路学里，电动势（英语：electromotive force，缩写为emf）表征一些电路元件供应电能的特性。这些电路元件称为「电动势源」。电化电池、太阳能电池、燃料电池、热电装置、发电机等等，都是电动势源。电动势源所供应的能量每单位电荷是其电动势。假设，电荷'},
 {'id': 1,
  'title': '楚河州',
  'context': '楚河州包括有整个楚河河谷及邻近的山脉与峡谷。河谷的黑土非常肥沃，而且被从楚河引来的河水灌溉着。当地的农业生产计有：小麦、玉蜀黍、甜菜、马铃薯、紫花苜蓿及各种不同品种的蔬菜及水果。在苏联统治期间，省内有不少农产品加工及其他工业，使省内涌现多个新市镇，如：托克马克、坎特（Kant）及卡拉巴尔塔（Kara-Balta）等。相对于国内其他省份，本州的人口成份比较复杂，计有：俄罗斯人、乌克兰人、东干人（中国回民的后裔）、朝鲜人及德国人等。'},
 {'id': 2,
  'title': '凱提文',
  'context': '凯提文(Kaithi，कैथी)，也叫做Kayathi或Kayasthi，是历史上的一种文字，曾广泛用于北印度，主要是以前的西北行省和Oudh（今天的北方邦）和比哈尔。它曾用于书写法律、行政和私人记录。Unicode技术委员会已经接受了在Unicode标准中编码凯提文的提案，范围是U+11080-110CF。'},
 {'id': 3,
  'title': '凱提文',
  'context': '用凯提文记录的文档可追溯到至少16世纪。这种文字广泛用在莫卧儿帝国期间。在1880年代英属印度期间，这种文字被认可为比哈尔邦法庭上的官方文字。尽管一般而言凯提文曾在某些地区比城文更加广泛使用，它现在已经失去了竞争力。'},
 {'id': 4,
  'title': '爱丽丝梦游仙境',
  'context': '第五章：毛毛虫的建议（Advice from a Caterpillar）爱丽丝见到一棵蘑菇，上面坐著一条蓝色的毛虫。他抽著水烟，向爱丽丝探问起来。爱丽丝回应他，自己正在个性转变期之中，时常心绪不宁，她甚至连一首诗都记不起来。毛虫离开之前，告诉了她蘑菇的秘密：吃其中一半会使她变高，吃另一半会使她变矮。于是，她

In [114]:
@contextmanager
def suppress_stdout_stderr():
    """A context manager that redirects stdout and stderr to devnull."""
    with open(os.devnull, 'w') as fnull:
        old_stdout, old_stderr = sys.stdout, sys.stderr
        sys.stdout, sys.stderr = fnull, fnull
        try:
            yield
        finally:
            sys.stdout, sys.stderr = old_stdout, old_stderr

keywords = []
for c in tqdm(contexts):
    try:
        # Try method one to extract keywords
        with suppress_stdout_stderr():
            extracted_keywords = kw_extractor.generate_keywords(c, top_k=5, rank_methods="mmr")[0]
    except ValueError:
        # If a ValueError is encountered, use method two to extract the keywords
        extracted_keywords = [item[0] for item in jieba.analyse.extract_tags(c, topK=5, withWeight=True)]
    keywords.append(extracted_keywords)

  0%|          | 0/4538 [00:00<?, ?it/s]

**Save results**

In [186]:
assert len(contexts) == len(keywords)
len(contexts), len(keywords)

(4538, 4538)

**Add `title` to `keywords`**

In [241]:
strip_key_words = keywords.copy()

ids = [item['id'] for item in data_zh]
titles = [item['title'] for item in data_zh]
# Clean title
titles = [re.sub(r'\W+', '', title) for title in titles]

strip_key_words = [([titles[i]] + sublist) for i, sublist in enumerate(strip_key_words)]

In [242]:
assert len(ids) == len(strip_key_words)

print(f"Total num: {len(strip_key_words)}")

Total num: 4538


**Drop duplicate title and keywords**

In [243]:
unique_data = [list(dict.fromkeys(sublist)) for sublist in strip_key_words]

assert len(unique_data) == len(strip_key_words)
print(f"Total num: {len(unique_data)}")

Total num: 4538


**Output `with title` version**

In [244]:
documents = [
    {   
        "id": id, 
        "keywords": kw
    } 
    for id, kw in zip(ids, unique_data)
]

json_data = json.dumps(documents, indent=4)

In [246]:
with open('./data/Context_Keywords_with_title_MLQA_zh.json', 'w') as f:
    f.write(json_data)

In [20]:
file_path = './data/MLQA_Keywords/zh/Context_Keywords_with_title_MLQA_zh.json'

with open(file_path, 'r') as f:
    context_kw = json.load(f)
context_kw[:5]

[{'id': 0, 'keywords': ['電動勢', '电动势源', '元件', '电池', '电路学', '特性']},
 {'id': 1, 'keywords': ['楚河州', '农产品', '黑土', '灌溉', '苏联']},
 {'id': 2, 'keywords': ['凱提文', '文字', '北方邦', '印度', '委员会', '提案']},
 {'id': 3, 'keywords': ['凱提文', '文字', '帝国', '追溯到', '法庭', '使用']},
 {'id': 4, 'keywords': ['爱丽丝梦游仙境', '毛毛虫', '转变期', '秘密', '章', '蘑菇']}]

**Output `WITHOUT title` version**

In [22]:
documents = [
    {   
        "id": id, 
        "keywords": kw
    } 
    for id, kw in zip(ids, keywords)
]

json_data = json.dumps(documents, indent=4)

with open('./data/MLQA_Keywords/zh/Context_Keywords_MLQA_zh.json', 'w') as f:
    f.write(json_data)

In [23]:
file_path = './data/MLQA_Keywords/zh/Context_Keywords_MLQA_zh.json'

with open(file_path, 'r') as f:
    context_kw = json.load(f)
context_kw[:5]

[{'id': 0, 'keywords': ['电动势源', '元件', '电池', '电路学', '特性']},
 {'id': 1, 'keywords': ['农产品', '黑土', '灌溉', '苏联']},
 {'id': 2, 'keywords': ['文字', '北方邦', '印度', '委员会', '提案']},
 {'id': 3, 'keywords': ['文字', '帝国', '追溯到', '法庭', '使用']},
 {'id': 4, 'keywords': ['毛毛虫', '转变期', '秘密', '章', '蘑菇']}]

## Keywords for `Query`

In [7]:
file_path = './data/QA_ZH.json'

with open(file_path, 'r') as f:
    data_zh_qa = json.load(f)

print(f"Total num: {len(data_zh_qa)}")
data_zh_qa[:2]

Total num: 5129


[{'question': '各电化电池都能提供电动势？',
  'answers': [{'text': '电化电池', 'answer_start': 71}],
  'id': '465f3fb044b5c50a78a2e2f9bc94c424d1f7d039',
  'context_id': 0,
  'title': '電動勢'},
 {'question': '哪水体有助土地如此多产？',
  'answers': [{'text': '楚河', 'answer_start': 36}],
  'id': '1aee17dd937cc1043e3ff47c38396541fc3409e5',
  'context_id': 1,
  'title': '楚河州'}]

In [8]:
questions = [data['question'] for data in data_zh_qa]

In [9]:
keywords_weight = []

for c in tqdm(questions):
    extracted_keywords = jieba.analyse.extract_tags(c, topK=5, withWeight=True)
    keywords_weight.append(extracted_keywords)

  0%|          | 0/5129 [00:00<?, ?it/s]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/fq/njyvd6397pjf4rh3xt5tzbsw0000gn/T/jieba.cache
Loading model cost 0.302 seconds.
Prefix dict has been built successfully.


**Add title to keywords**

In [10]:
ids = [item['id'] for item in data_zh_qa]
context_ids = [item['context_id'] for item in data_zh_qa]
titles = [item['title'] for item in data_zh_qa]
# Clean title
titles = [re.sub(r'\W+', '', title) for title in titles]

assert len(ids) == len(keywords_weight)

keywords_list = [[kw[0] for kw in group] for group in keywords_weight]
prob_list = [[float(kw[1]) for kw in group] for group in keywords_weight]

updated_keywords_list = []
updated_prob_list = []
for i, (sublist, prob_sublist) in enumerate(zip(keywords_list, prob_list)):
    if not sublist:
        new_prob = 3.0
    else:
        new_prob = prob_sublist[0] + 0.1
        
        if titles[i] in sublist:
            # Find and delete duplicate words and prob
            index = sublist.index(titles[i])
            del sublist[index]
            del prob_sublist[index]

    updated_keywords_list.append([titles[i]] + sublist)
    updated_prob_list.append([new_prob] + prob_sublist)

assert len(updated_prob_list) == len(updated_keywords_list)
print(f"Total num: {len(updated_keywords_list)}")


Total num: 5129


```
from:

{'id': '56651aa8981b5a8270c94d87eed6e0032d605209',
  'context id': 21,
  'title': '结核病',
  'keywords': ['结核病', '粟粒状', '结核病', '年龄段', '哪个', '最好'],
  'prob': [2.49095350058, 2.39095350058, 2.02789550726, 1.986077147684, 1.360791254696, 1.15589888198]},

to:

{'id': '56651aa8981b5a8270c94d87eed6e0032d605209',
  'context id': 21,
  'title': '结核病',
  'keywords': ['结核病', '粟粒状', '年龄段', '哪个', '最好'],
  'prob': [2.49095350058, 2.39095350058, 1.986077147684, 1.360791254696, 1.15589888198]},
```

**Output `with title` version**

In [59]:
documents = [
    {   
        "id": id, 
        "context id": c_id,
        "keywords": kw,
        "prob" : p
    } 
    for id, c_id, kw, p in zip(ids, context_ids, updated_keywords_list, updated_prob_list)
]

json_data = json.dumps(documents, indent=4)

In [60]:
with open('./data/MLQA_Keywords/zh/Query_Keywords_with_title_MLQA_zh.json', 'w') as f:
    f.write(json_data)

In [71]:
file_path = './data/MLQA_Keywords/zh/Query_Keywords_with_title_MLQA_zh.json'

with open(file_path, 'r') as f:
    query_kw = json.load(f)
query_kw[:2]

[{'id': '465f3fb044b5c50a78a2e2f9bc94c424d1f7d039',
  'context id': 0,
  'keywords': ['電動勢', '电化', '电动势', '电池', '提供'],
  'prob': [2.8390596682,
   2.7390596682,
   2.376557124325,
   1.8797154086425,
   1.17103910523]},
 {'id': '1aee17dd937cc1043e3ff47c38396541fc3409e5',
  'context id': 1,
  'keywords': ['楚河州', '多产', '有助', '水体', '土地', '如此'],
  'prob': [2.1579519478600004,
   2.0579519478600004,
   1.8714765739419998,
   1.69960005402,
   1.048570666164,
   0.9844538674700001]}]

**Output `WITHOUT title` version**

In [27]:
documents = [
    {   
        "id": id, 
        "context id": c_id,
        "keywords": kw,
        "prob" : p
    } 
    for id, c_id, kw, p in zip(ids, context_ids, keywords_list, prob_list)
]

json_data = json.dumps(documents, indent=4)

In [28]:
with open('./data/MLQA_Keywords/zh/Query_Keywords_MLQA_zh.json', 'w') as f:
    f.write(json_data)

In [29]:
file_path = './data/MLQA_Keywords/zh/Query_Keywords_MLQA_zh.json'

with open(file_path, 'r') as f:
    query_kw = json.load(f)
query_kw[:2]

[{'id': '465f3fb044b5c50a78a2e2f9bc94c424d1f7d039',
  'context id': 0,
  'keywords': ['电化', '电动势', '电池', '提供'],
  'prob': [2.7390596682, 2.376557124325, 1.8797154086425, 1.17103910523]},
 {'id': '1aee17dd937cc1043e3ff47c38396541fc3409e5',
  'context id': 1,
  'keywords': ['多产', '有助', '水体', '土地', '如此'],
  'prob': [2.0579519478600004,
   1.8714765739419998,
   1.69960005402,
   1.048570666164,
   0.9844538674700001]}]